In [1]:
import collections
import tensorflow as tf
from datetime import datetime
import time
import math
slim = tf.contrib.slim

# collections.namedtuple block class

In [2]:
class Block(collections.namedtuple('Block', ['scope', 'unit_fn', 'args'])):
    'A named tuple describing a ResNet block.'

# subsample function

In [3]:
def subsample(inputs, factor, scope=None):
    if factor == 1:
        return inputs
    else:
        return slim.max_pool2d(inputs, [1, 1], stride=factor, scope=scope)

# conv layer function

In [4]:
def conv2d_same(inputs, num_outputs, kernel_size, stride, scope=None):
    if stride == 1:
        return slim.conv2d(inputs, num_outputs, kernel_size, stride=1, padding='SAME', scope=scope)
    else:
        pad_total = kernel_size - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end], [pad_beg, pad_end], [0, 0]])
        return slim.conv2d(inputs, num_outputs, kernel_size, stride=stride, padding='VALID', scope=scope)

# blocks function

In [5]:
@slim.add_arg_scope
def stack_blocks_dense(net, blocks, outputs_collections=None):
    for block in blocks:
        with tf.variable_scope(block.scope, 'block', [net]) as sc:
            for i, unit in enumerate(block.args):
                with tf.variable_scope('unit_%d' % (i+1), values=[net]):
                    unit_depth, unit_depth_bottleneck, unit_stride=unit
                    net = block.unit_fn(net, depth=unit_depth,
                                       depth_bottleneck=unit_depth_bottleneck,
                                       stride=unit_stride)
            net = slim.utils.collect_named_outputs(outputs_collections, sc.name, net)
    return net

# arg_scope function

In [6]:
def resnet_arg_scope(is_training=True,
                                    weight_decay=0.0001,
                                    batch_norm_decay=0.997,
                                    batch_norm_epsilon=1e-5,
                                    batch_norm_scale=True):
    
    batch_norm_params = {
        'is_training': is_training,
        'decay': batch_norm_decay,
        'epsilon': batch_norm_epsilon,
        'scale': batch_norm_scale,
        'updates_collections': tf.GraphKeys.UPDATE_OPS,}
    
    with slim.arg_scope([slim.conv2d],
                        weights_regularizer = slim.l2_regularizer(weight_decay),
                        weights_initializer = slim.variance_scaling_initializer(),
                        activation_fn = tf.nn.relu,
                        normalizer_fn = slim.batch_norm,
                        normalizer_params = batch_norm_params):
        with slim.arg_scope([slim.batch_norm], **batch_norm_params):
            with slim.arg_scope([slim.max_pool2d], padding='SAME') as arg_sc:
                return arg_sc

# bottleneck residual unit function

In [7]:
@slim.add_arg_scope
def bottleneck(inputs, depth, depth_bottleneck, stride,
              outputs_collections=None, scope=None):
    with tf.variable_scope(scope, 'bottleneck_v2', [inputs]) as sc:
        depth_in = slim.utils.last_dimension(inputs.get_shape(), min_rank=4)
        preact = slim.batch_norm(inputs, activation_fn=tf.nn.relu, scope='preact')
        if depth == depth_in:
            shortcut = subsample(inputs, stride, 'shortcut')
        else:
            shortcut = slim.conv2d(preact, depth, [1, 1], stride=stride,
                                  normalizer_fn=None, activation_fn=None, scope='shortcut')
            
        residual = slim.conv2d(preact, depth_bottleneck, [1, 1], stride=1, scope='conv1')
        residual = conv2d_same(preact, depth_bottleneck, 3, stride, scope='conv2')
        residual = slim.conv2d(residual, depth, [1, 1], stride=1,
                              normalizer_fn=None, activation_fn=None, scope='conv3')
        
        output = shortcut + residual
        return slim.utils.collect_named_outputs(outputs_collections, sc.name, output)

# resnet v2 main function

In [8]:
def resnet_v2(inputs,
                       blocks,
                       num_classes=None,
                       global_pool=True,
                       include_root_block=True,
                       reuse=None,
                       scope=None):
    with tf.variable_scope(scope, 'resnet_v2', [inputs], reuse=reuse) as sc:
        end_points_collection = sc.original_name_scope + '_end_points'
        with slim.arg_scope([slim.conv2d, bottleneck, stack_blocks_dense], 
                                           outputs_collections=end_points_collection):
            net = inputs
            if include_root_block:
                with slim.arg_scope([slim.conv2d], activation_fn=None, normalizer_fn=None):
                    net = conv2d_same(net, 64, 7, stride=2, scope='conv1')
                net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1')
            net = stack_blocks_dense(net, blocks)
            net = slim.batch_norm(net, activation_fn=tf.nn.relu, scope='postnorm')
            if global_pool:
                net = tf.reduce_mean(net, [1, 2], name='pool5', keep_dims=True)
            if num_classes is not None:
                net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None, normalizer_fn=None, scope='logits')
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if num_classes is not None:
                end_points['predictions'] = slim.softmax(net, scope='predictions')
            return net, end_points

# resnet 18-layer function

In [9]:
def resnet_v2_18(inputs,
                            num_classes=None,
                            global_pool=True,
                            reuse=None,
                            scope='resnet_v2_18'):
    blocks = [
        Block('block1', bottleneck, [(64, 2)] * 2),
        Block('block2', bottleneck, [(128, 2)] * 2),
        Block('block3', bottleneck, [(256, 2)] * 2),
        Block('block4', bottleneck, [(512, 2)] * 2)]
    return resnet_v2(intpus, blocks, num_classes, global_pool,
                    include_root_block=True, reuse=reuse, scope=scope)

# resnet 34-layer function

In [10]:
def resnet_v2_34(inputs,
                            num_classes=None,
                            global_pool=True,
                            reuse=None,
                            scope='resnet_v2_34'):
    blocks = [
        Block('block1', bottleneck, [(64, 2)] * 3),
        Block('block2', bottleneck, [(128, 2)] * 4),
        Block('block3', bottleneck, [(256, 2)] * 6),
        Block('block4', bottleneck, [(512, 2)] * 3)]
    return resnet_v2(intpus, blocks, num_classes, global_pool,
                    include_root_block=True, reuse=reuse, scope=scope)

# resnet 50 layers function

In [11]:
def resnet_v2_50(inputs,
                            num_classes=None,
                            global_pool=True,
                            reuse=None,
                            scope='resnet_v2_50'):
    blocks = [
        Block('block1', bottleneck, [(256, 64, 1)] * 2 + [(256, 64, 2)]),
        Block('block2', bottleneck, [(512, 128, 1)] * 3 + [(512, 128, 2)]),
        Block('block3', bottleneck, [(1024, 256, 1)] * 5 + [(1024, 256, 2)]),
        Block('block4', bottleneck, [(2048, 512, 1)] * 3)]
    return resnet_v2(intpus, blocks, num_classes, global_pool,
                    include_root_block=True, reuse=reuse, scope=scope)

# resnet 101 layers function

In [12]:
def resnet_v2_101(inputs,
                              num_classes=None,
                              global_pool=True,
                              reuse=None,
                              scope='resnet_v2_101'):
    blocks = [
        Block('block1', bottleneck, [(256, 64, 1)] * 2 + [(256, 64, 2)]),
        Block('block2', bottleneck, [(512, 128, 1)] * 3 + [(512, 128, 2)]),
        Block('block3', bottleneck, [(1024, 256, 1)] * 22 + [(1024, 256, 2)]),
        Block('block4', bottleneck, [(2048, 512, 1)] * 3)]
    return resnet_v2(intpus, blocks, num_classes, global_pool,
                    include_root_block=True, reuse=reuse, scope=scope)

# resnet 152 layers function

In [13]:
def resnet_v2_152(inputs,
                              num_classes=None,
                              global_pool=True,
                              reuse=None,
                              scope='resnet_v2_152'):
    blocks = [
        Block('block1', bottleneck, [(256, 64, 1)] * 2 + [(256, 64, 2)]),
        Block('block2', bottleneck, [(512, 128, 1)] * 7 + [(512, 128, 2)]),
        Block('block3', bottleneck, [(1024, 256, 1)] * 35 + [(1024, 256, 2)]),
        Block('block4', bottleneck, [(2048, 512, 1)] * 3)]
    return resnet_v2(inputs, blocks, num_classes, global_pool,
                    include_root_block=True, reuse=reuse, scope=scope)

# resnet 200 layers function

In [14]:
def resnet_v2_200(inputs,
                              num_classes=None,
                              global_pool=True,
                              reuse=None,
                              scope='resnet_v2_200'):
    blocks = [
        Block('block1', bottleneck, [(256, 64, 1)] * 2 + [(256, 64, 2)]),
        Block('block2', bottleneck, [(512, 128, 1)] * 23 + [(512, 128, 2)]),
        Block('block3', bottleneck, [(1024, 256, 1)] * 35 + [(1024, 256, 2)]),
        Block('block4', bottleneck, [(2048, 512, 1)] * 3)]
    return resnet_v2(intpus, blocks, num_classes, global_pool,
                    include_root_block=True, reuse=reuse, scope=scope)

# time run function

In [15]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s: step %d, duration = %.3f' %
                     (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print('%s: %s across %d steps, %.3f +/- %.3f sec/batch' %
         (datetime.now(), info_string, num_batches, mn, sd))

# train

In [16]:
batch_size = 32
height, width = 224, 224
inputs = tf.random_uniform((batch_size, height, width, 3))
with slim.arg_scope(resnet_arg_scope(is_training=False)):
    net, end_points = resnet_v2_152(inputs, 1000)
    
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
num_batches=100
time_tensorflow_run(sess, net, "Forward")

2017-06-25 12:54:39.806201: step 0, duration = 47.910
2017-06-25 13:02:38.213352: step 10, duration = 47.589
2017-06-25 13:10:33.027000: step 20, duration = 47.741
2017-06-25 13:18:35.709489: step 30, duration = 48.911
2017-06-25 13:26:40.459215: step 40, duration = 48.261
2017-06-25 13:34:41.137442: step 50, duration = 48.620
2017-06-25 13:42:50.678212: step 60, duration = 47.413
2017-06-25 13:50:47.643907: step 70, duration = 47.687
2017-06-25 13:58:33.353348: step 80, duration = 46.508
2017-06-25 14:06:22.141687: step 90, duration = 46.954
2017-06-25 14:13:28.280838: Forward across 100 steps, 47.764 +/- 1.056 sec/batch
